In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

2023-12-28 15:08:56.396520: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-28 15:08:56.396541: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-28 15:08:56.397366: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-28 15:08:56.401468: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-28 15:08:56.898371: W tensorflow/compiler/tf2

In [2]:
class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.dense_dim = dense_dim
        self.attention = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.dense_projection = keras.Sequential([layers.Dense(dense_dim, activation="relu"), layers.Dense(embed_dim)])
        self.layer_norm1 = layers.LayerNormalization()
        self.layer_norm2 = layers.LayerNormalization()
        
    def call(self, inputs, mask=None):
        if mask is not None:
            mask = mask[:, tf.newaxis, :]
        attention_output = self.attention(inputs, inputs, attention_mask=mask)
        projection_input = self.layer_norm1(inputs + attention_output) 
        projection_output = self.dense_projection(projection_input)
        return self.layer_norm2(projection_output + projection_input)
    
    def get_config(self):
        config = super().get_config()
        config.update({
            'embed_dim': self.embed_dim,
            'num_heads': self.num_heads,
            'dense_dim': self.dense_dim,
        })
        return config

In [4]:
vocab_size = 20000
embed_dim = 256
num_heads = 2
dense_dim = 32

inputs = layers.Input(shape=(None,), dtype='int64')
x = layers.Embedding(vocab_size, embed_dim)(inputs)
x = TransformerEncoder(embed_dim, dense_dim, num_heads)(x)
x = layers.GlobalAveragePooling1D()(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation='sigmoid')(x)
model = keras.Model(inputs=inputs, outputs=outputs)
model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

In [8]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding_1 (Embedding)     (None, None, 256)         5120000   
                                                                 
 transformer_encoder_1 (Tra  (None, None, 256)         543776    
 nsformerEncoder)                                                
                                                                 
 global_average_pooling1d (  (None, 256)               0         
 GlobalAveragePooling1D)                                         
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_2 (Dense)             (None, 1)                 257   

In [10]:
import pathlib

max_length = 600
max_tokens = 20000

batch_size = 32

base_dir = pathlib.Path("./aclImdb")
train_dir = base_dir / "train"
val_dir = base_dir / "val"
test_dir = base_dir / "test"

train_ds = keras.utils.text_dataset_from_directory(
    train_dir, batch_size=batch_size
)
val_ds = keras.utils.text_dataset_from_directory(
    val_dir, batch_size=batch_size
)
test_ds = keras.utils.text_dataset_from_directory(
    test_dir, batch_size=batch_size
)

text_only_train_ds = train_ds.map(lambda x, y: x)

text_vectorization = layers.TextVectorization(
    max_tokens=max_tokens,
    output_sequence_length=max_length,
    output_mode="int"
)

text_vectorization.adapt(text_only_train_ds)

int_train_ds = train_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls=4)
int_val_ds = val_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls=4)
int_test_ds = test_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls=4)

Found 20000 files belonging to 2 classes.
Found 5000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.


In [ ]:
callbacks = [keras.callbacks.ModelCheckpoint("transformer_encoder.keras", save_best_only=True)]
model.fit(int_train_ds, validation_data=int_val_ds, epochs=20, callbacks=callbacks)

In [12]:
model = keras.models.load_model("transformer_encoder.keras", custom_objects={"TransformerEncoder": TransformerEncoder})
model.evaluate(int_test_ds)

782/782 [==============================] - 7s 9ms/step - loss: 0.3001 - accuracy: 0.8789


[0.300081342458725, 0.8789200186729431]

In [5]:
class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, input_dim, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.token_embedding = layers.Embedding(input_dim=input_dim, output_dim=output_dim)
        self.position_embedding = layers.Embedding(input_dim=sequence_length, output_dim=output_dim)
        self.sequence_length = sequence_length
        self.input_dim = input_dim
        self.output_dim = output_dim
        
    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embedding(inputs)
        embedded_positions = self.position_embedding(positions)
        embedding = embedded_tokens + embedded_positions
        return embedding
    
    def compute_mask(self, inputs, mask=None):
        return tf.math.not_equal(inputs, 0)
    
    def get_config(self):
        config = super().get_config()
        config.update({
            "output_dim": self.output_dim,
            "sequence_length": self.sequence_length,
            "input_dim": self.input_dim
        })
        return config

In [16]:
vocab_size = 20000
sequence_length = 600
embedding_dim = 256
num_heads = 2
dense_dim = 32

inputs  = keras.Input(shape=(None,), dtype='int64')
x = PositionalEmbedding(sequence_length, vocab_size, embedding_dim)(inputs)
x = TransformerEncoder(embedding_dim, dense_dim, num_heads)(x)
x = layers.GlobalAveragePooling1D()(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation='sigmoid')(x)
model = keras.Model(inputs=inputs, outputs=outputs)
model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

In [17]:
model.summary()

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, None)]            0         
                                                                 
 positional_embedding_2 (Po  (None, None, 256)         5273600   
 sitionalEmbedding)                                              
                                                                 
 transformer_encoder_4 (Tra  (None, None, 256)         543776    
 nsformerEncoder)                                                
                                                                 
 global_average_pooling1d_4  (None, 256)               0         
  (GlobalAveragePooling1D)                                       
                                                                 
 dropout_4 (Dropout)         (None, 256)               0         
                                                           

In [18]:
callbacks = [keras.callbacks.ModelCheckpoint("transformer_encoder_positional_emb.keras", save_best_only=True)]
history = model.fit(int_train_ds, validation_data=int_val_ds, epochs=20, callbacks=callbacks)

Epoch 1/20
625/625 [==============================] - 25s 39ms/step - loss: 0.5255 - accuracy: 0.7372 - val_loss: 0.3345 - val_accuracy: 0.8674
Epoch 2/20
625/625 [==============================] - 19s 31ms/step - loss: 0.3032 - accuracy: 0.8778 - val_loss: 0.3744 - val_accuracy: 0.8496
Epoch 3/20
625/625 [==============================] - 17s 28ms/step - loss: 0.2421 - accuracy: 0.9046 - val_loss: 0.3478 - val_accuracy: 0.8702
Epoch 4/20
625/625 [==============================] - 16s 26ms/step - loss: 0.2089 - accuracy: 0.9189 - val_loss: 0.3830 - val_accuracy: 0.8814
Epoch 5/20
625/625 [==============================] - 17s 27ms/step - loss: 0.1767 - accuracy: 0.9336 - val_loss: 0.3338 - val_accuracy: 0.8820
Epoch 6/20
625/625 [==============================] - 16s 26ms/step - loss: 0.1558 - accuracy: 0.9420 - val_loss: 0.4537 - val_accuracy: 0.8894
Epoch 7/20
625/625 [==============================] - 16s 26ms/step - loss: 0.1365 - accuracy: 0.9490 - val_loss: 0.4912 - val_accuracy:

In [19]:
model = keras.models.load_model("transformer_encoder_positional_emb.keras", 
                                custom_objects={
                                    'TransformerEncoder': TransformerEncoder, 
                                    'PositionalEmbedding': PositionalEmbedding
                                })
model.evaluate(int_test_ds)

782/782 [==============================] - 8s 10ms/step - loss: 0.3365 - accuracy: 0.8715


[0.3364552855491638, 0.8714799880981445]